In [1]:
import tensorflow as tf
import os
import numpy as np
from glob import glob as glb
from tqdm import *
import h5py
from tensorflow.contrib import rnn

min_queue_examples = 1000

/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p


In [2]:
def load_flow(filename, shape):
  stream_flow = h5py.File(filename, 'r')
  flow_state_vel = np.array(stream_flow['Velocity_0'][:])
  flow_state_vel = flow_state_vel.reshape([shape[0], shape[1]+128, 3])[0:shape[0],0:shape[1],0:2]
  stream_flow.close()
  return flow_state_vel

def load_boundary(filename, shape):
  stream_boundary = h5py.File(filename, 'r')
  boundary_cond = np.array(stream_boundary['Gamma'][:])
  boundary_cond = boundary_cond.reshape([shape[0], shape[1]+128, 1])[0:shape[0],0:shape[1],:]
  stream_boundary.close()
  return boundary_cond

def read_data(filename_queue, shape):
  reader = tf.TFRecordReader()
  key, serialized_example = reader.read(filename_queue)
  features = tf.parse_single_example(
    serialized_example,
    features={
      'boundary':tf.FixedLenFeature([],tf.string),
      'sflow':tf.FixedLenFeature([],tf.string),
      'vmax':tf.FixedLenFeature([],tf.int64),
    }) 
  boundary = tf.decode_raw(features['boundary'], tf.uint8)
  sflow = tf.decode_raw(features['sflow'], tf.float32)
  vmax = tf.cast(features['vmax'], tf.int32)

  boundary = tf.reshape(boundary, [shape[0], shape[1], 1])
  sflow = tf.reshape(sflow, [shape[0], shape[1], 2])
  boundary = tf.to_float(boundary)
  sflow = tf.to_float(sflow)

  return boundary, sflow, vmax

def _generate_image_label_batch(boundary, sflow, vmax, batch_size, shuffle=True):
  num_preprocess_threads = 1
  #Create a queue that shuffles the examples, and then
  #read 'batch_size' images + labels from the example queue.
  boundarys, sflows, vmax = tf.train.shuffle_batch(
    [boundary, sflow, vmax],
    batch_size=batch_size,
    num_threads=num_preprocess_threads,
    capacity=min_queue_examples + 3 * batch_size,
    min_after_dequeue=min_queue_examples)
  return boundarys, sflows, vmax

def flow_inputs(batch_size):
  shape = (128,256)

  tfrecord_filename = glb('../data/*.tfrecords') 
  
  filename_queue = tf.train.string_input_producer(tfrecord_filename) 

  boundary, sflow,vmax = read_data(filename_queue, shape)

  boundarys, sflows, vmax = _generate_image_label_batch(boundary, sflow,vmax, batch_size)

  return boundarys, sflows, vmax

In [3]:
n_batch = 8
learning_rate = 0.0001
keep_prob = 0.7
n_hidden = 128

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32 , [None,128,256,1], name="X")

boundary, sflow, vmax = flow_inputs(n_batch)
X = boundary

v = tf.placeholder(tf.float32 ,vmax.shape, name="v")
v = tf.to_float(vmax)

print(v)

#Conv1
W1 = tf.Variable(tf.random_normal([16, 16, 1, 128], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 8, 16, 1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)

Tensor("ToFloat_2:0", shape=(8,), dtype=float32)


In [4]:
#Cov2
W2 = tf.Variable(tf.random_normal([4,4,128,512], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 4, 4, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

In [5]:
L2_flat = tf.reshape(L2,[-1,4*4*512])

In [6]:
W3 = tf.get_variable("W", shape=[512 * 4 * 4, 1024],
                     initializer=tf.contrib.layers.xavier_initializer())

In [7]:
b3 = tf.Variable(tf.random_normal([1024]))
L3 = tf.nn.relu(tf.matmul(L2_flat, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
print(L3)
# vmax를 state로 주고 input으로 L3를 RNN에 넣은다.

Tensor("dropout_2/mul:0", shape=(8, 1024), dtype=float32)


In [8]:
# RNN layer

WR = tf.Variable(tf.random_normal([n_hidden , 1024], stddev=0.01))
BR = tf.Variable(tf.random_normal([1024]))

L3 = tf.reshape(L3 , [n_batch,1024,1])
state = v
cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_hidden,reuse=tf.AUTO_REUSE)
model, states = tf.nn.dynamic_rnn(cell=cell, inputs=L3, dtype=tf.float32, time_major=False)
model = tf.layers.dense(model,units=1,activation=None)
model = tf.nn.relu(model)
L3 = tf.reshape(model,[n_batch,1024])

In [9]:
# L3 = tf.divide(L3,y)
S1, S2 = tf.split(L3, [512, 512], 1)

In [10]:
S1= tf.reshape(S1,[n_batch,1,1,512])
S2= tf.reshape(S2,[n_batch,1,1,512])

In [11]:
# deconv1
W4_1 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_1 = tf.nn.conv2d_transpose(S1,W4_1,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_1 = tf.nn.relu(L4_1)
L4_1 = tf.nn.dropout(L4_1, keep_prob)

W4_2 = tf.Variable(tf.random_normal([8,8,512,512], stddev=0.01))
L4_2 = tf.nn.conv2d_transpose(S2,W4_2,output_shape=[n_batch,8,8,512],strides=[1,8, 8, 1], padding='SAME')
L4_2 = tf.nn.relu(L4_2)
L4_2 = tf.nn.dropout(L4_2, keep_prob)

In [12]:
## deconv2
W5_1 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_1 = tf.nn.conv2d_transpose(L4_1,W5_1,output_shape=[n_batch,32,64,256],strides=[1, 4, 8, 1], padding='SAME')
L5_1 = tf.nn.relu(L5_1)
L5_1 = tf.nn.dropout(L5_1, keep_prob)

W5_2 = tf.Variable(tf.random_normal([4,8,256,512], stddev=0.01))
L5_2 = tf.nn.conv2d_transpose(L4_2,W5_2,output_shape=[n_batch,32,64,256],strides=[1,4, 8, 1], padding='SAME')
L5_2 = tf.nn.relu(L5_2)
L5_2 = tf.nn.dropout(L5_2, keep_prob)

In [13]:
# deconv3
W6_1 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_1 = tf.nn.conv2d_transpose(L5_1,W6_1,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_1 = tf.nn.relu(L6_1)
L6_1 = tf.nn.dropout(L6_1, keep_prob)

W6_2 = tf.Variable(tf.random_normal([2,2,32,256], stddev=0.01))
L6_2 = tf.nn.conv2d_transpose(L5_2,W6_2,output_shape=[n_batch,64,128,32],strides=[1,2, 2, 1], padding='SAME')
L6_2 = tf.nn.relu(L6_2)
L6_2 = tf.nn.dropout(L6_2, keep_prob)

In [14]:
# deconv4
W7_1 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_1 = tf.nn.conv2d_transpose(L6_1,W7_1,output_shape=[n_batch,128,256,1],strides=[1,2,2, 1], padding='SAME')
L7_1 = tf.nn.dropout(L7_1, keep_prob)

W7_2 = tf.Variable(tf.random_normal([2,2,1,32], stddev=0.01))
L7_2 = tf.nn.conv2d_transpose(L6_2,W7_2,output_shape=[n_batch,128,256,1],strides=[1,2, 2, 1], padding='SAME')
L7_2 = tf.nn.dropout(L7_2, keep_prob)

In [15]:
sflow_p = tf.stack([L7_1 , L7_2] , axis=3)
sflow_p = tf.reshape(sflow_p , [n_batch,128,256,2])
# loss = tf.reduce_mean(tf.square(sflow_p - sflow))
loss = tf.nn.l2_loss(sflow_p - sflow)

In [16]:
#loss 
total_loss = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [17]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model_save')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

INFO:tensorflow:Restoring parameters from ./model_save/model.ckpt


In [18]:
tf.train.start_queue_runners(sess=sess)

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 123145444679680)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145448886272)>]

In [19]:
# v = sess.run([v],feed_dict={})
# print(v)

In [ ]:
steps = 200

for epoch in range(steps):    
  total_cost = 0
  _, cost_val = sess.run([total_loss, loss],feed_dict={})
  total_cost += cost_val

  print('Epoch:', '%04d' % (epoch + 1),
        'Avg. cost =', '{:f}'.format(total_cost))
print("최적화 완료") 
saver.save(sess, "./model_save/model.ckpt")

('Epoch:', '0001', 'Avg. cost =', '610.915710')
('Epoch:', '0002', 'Avg. cost =', '458.887634')
('Epoch:', '0003', 'Avg. cost =', '628.047363')
('Epoch:', '0004', 'Avg. cost =', '858.528625')
('Epoch:', '0005', 'Avg. cost =', '616.375061')
('Epoch:', '0006', 'Avg. cost =', '506.004272')
('Epoch:', '0007', 'Avg. cost =', '858.511047')
('Epoch:', '0008', 'Avg. cost =', '914.012451')
('Epoch:', '0009', 'Avg. cost =', '842.890503')
('Epoch:', '0010', 'Avg. cost =', '461.316803')
('Epoch:', '0011', 'Avg. cost =', '571.886230')
('Epoch:', '0012', 'Avg. cost =', '146.558914')
('Epoch:', '0013', 'Avg. cost =', '887.718750')
('Epoch:', '0014', 'Avg. cost =', '499.093231')
('Epoch:', '0015', 'Avg. cost =', '446.360657')
('Epoch:', '0016', 'Avg. cost =', '661.914673')
('Epoch:', '0017', 'Avg. cost =', '600.225220')
('Epoch:', '0018', 'Avg. cost =', '653.355103')
('Epoch:', '0019', 'Avg. cost =', '703.898560')
('Epoch:', '0020', 'Avg. cost =', '438.500793')
('Epoch:', '0021', 'Avg. cost =', '600.8

Exception in thread QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue:
Traceback (most recent call last):
  File "/Users/ichang-geon/.pyenv/versions/2.7.14/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/ichang-geon/.pyenv/versions/2.7.14/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/tensorflow/python/training/queue_runner_impl.py", line 252, in _run
    enqueue_callable()
  File "/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1244, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "/Users/ichang-geon/.pyenv/versions/2.7.14/envs/flow/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 1409, in _call_tf_sessionrun
    run_metadat

('Epoch:', '0165', 'Avg. cost =', '824.946045')
('Epoch:', '0166', 'Avg. cost =', '718.452026')
('Epoch:', '0167', 'Avg. cost =', '748.299133')
